In [ ]:
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
tf.random.set_seed(2382)
colormap = {0:'black', 1:'red', 2:'blue'}

# 0. Load the data

In [ ]:
data_raw = load_iris(as_frame=True)
species_names = data_raw['target_names']
data = data_raw['frame']
data = data.drop(columns=['sepal length (cm)','sepal width (cm)'])
data = data.rename(columns={'petal length (cm)':'petal_length','petal width (cm)':'petal_width','target':'species'})

X = data[['petal_length','petal_width']].to_numpy()
y = data['species'].to_numpy().reshape(-1, 1)

data['species'] = data['species'].map({0:'setosa',1:'versicolor',2:'virginica'})
data

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
for yy in range(3):
    ind = y==yy
    ax.scatter(data.loc[ind,'petal_length'],data.loc[ind,'petal_width'],label=species_names[yy],c=colormap[yy])
ax.legend(fontsize=14)
ax.set_xlabel('petal length',fontsize=14)
ax.set_ylabel('petal width',fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

xlim = ax.get_xlim()
ylim = ax.get_ylim()

# 1. Output encoding

In [ ]:
y_ohe = tf.keras.utils.to_categorical(y)
y_ohe

# 2. Neural network model

In [ ]:
model = Sequential([
    Dense(40, input_dim=2, activation='relu'),
    Dense(3, activation="softmax") ])

model.compile(loss="categorical_crossentropy", 
              metrics=["accuracy"])

# 3. Train the model

In [ ]:
history = model.fit(X, y_ohe, 
                    epochs=500)

### Evolution of the model during training

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(history.history['accuracy'], linewidth=2)
ax.grid(linestyle='--')
ax.set_xlabel('epochs',fontsize=14)
ax.set_ylabel('accuracy',fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

### Plot decistion boundaries

In [ ]:
d1_grid, d2_grid = np.meshgrid(np.arange(xlim[0], xlim[1], 0.02), np.arange(ylim[0], ylim[1], 0.02))
X0 = d1_grid.ravel()
X1 = d2_grid.ravel()

d12_array = np.empty((len(X0),2))
d12_array[:,0] = X0
d12_array[:,1] = X1
i=2

y_array = model.predict(d12_array)
y_grid = y_array.argmax(axis=1).reshape(d1_grid.shape)


In [ ]:

fig, ax = plt.subplots()
plt.contourf(d1_grid, d2_grid, y_grid,alpha=0.3,levels=3)
for yy in range(3):
    ind = y==yy
    ax.scatter(data.loc[ind,'petal_length'],data.loc[ind,'petal_width'],label=species_names[yy],c=colormap[yy])
ax.legend()
ax.set_xlabel('petal length',fontsize=14)
ax.set_ylabel('petal width',fontsize=14)
ax.spines[:].set_visible(False)